## Exercises

Using the titanic data, in your classification-exercises repository, create a notebook, model.ipynb where you will do the following:

In [1]:
import numpy as np
import pandas as pd
import acquire
import sys
import env
##############
# Note that this prepare is modified to reflect the specific work done in the Prepare Lesson
#############

import prepare
from sklearn.tree import DecisionTreeClassifier , export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
import graphviz
from graphviz import Graph

In [2]:
# acquire the df
df = acquire.get_titanic_data()

In [3]:
# peek at the df
df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [4]:
df = df[df.age.isna() == False]

In [5]:
df['is_male'] = df.sex == 'male'
dummy_df = pd.get_dummies(df[['class','embark_town']])
dummy_df

,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,0,1,0,0,1
1,1,0,0,1,0,0
2,0,0,1,0,0,1
3,1,0,0,0,0,1
4,0,0,1,0,0,1
...,...,...,...,...,...,...
885,0,0,1,0,1,0
886,0,1,0,0,0,1
887,1,0,0,0,0,1
889,1,0,0,1,0,0


In [6]:
df = pd.concat([df, dummy_df],axis=1)

In [7]:
#drop columns
df.drop(columns=['passenger_id','pclass','embarked','sex','deck','embark_town','class','is_male'],inplace=True)
df.head()

,survived,age,sibsp,parch,fare,alone,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,22.0,1,0,7.2500,0,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,0,1,0,0,1,0,0
2,1,26.0,0,0,7.9250,1,0,0,1,0,0,1
3,1,35.0,1,0,53.1000,0,1,0,0,0,0,1
4,0,35.0,0,0,8.0500,1,0,0,1,0,0,1


In [8]:
df.head()

,survived,age,sibsp,parch,fare,alone,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,22.0,1,0,7.2500,0,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,0,1,0,0,1,0,0
2,1,26.0,0,0,7.9250,1,0,0,1,0,0,1
3,1,35.0,1,0,53.1000,0,1,0,0,0,0,1
4,0,35.0,0,0,8.0500,1,0,0,1,0,0,1


In [9]:
# split the data
train, validate, test = prepare.train_validate_test_split(df)

In [10]:
train.head()

,survived,age,sibsp,parch,fare,alone,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
652,0,21.0,0,0,8.4333,1,0,0,1,0,0,1
813,0,6.0,4,2,31.2750,0,0,0,1,0,0,1
194,1,44.0,0,0,27.7208,1,1,0,0,1,0,0
417,1,18.0,0,2,13.0000,0,0,1,0,0,0,1
460,1,48.0,0,0,26.5500,1,1,0,0,0,0,1


In [11]:
# We will be attempting to make a Decision Tree Classifier Model that will predict survival on the 
# Titanic that performs better than the baseline

1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [12]:
# obtain our mode
train.survived.value_counts()

0    237
1    162
Name: survived, dtype: int64

In [13]:
train['baseline_assumption_death'] = 0

In [14]:
print(f'Our baseline accuracy for nonsurvival in all cases on the Titanic Dataset is {(train.baseline_assumption_death == train.survived).mean():.3}')

Our baseline accuracy for nonsurvival in all cases on the Titanic Dataset is 0.594


2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [15]:
# create the model
clf1 = DecisionTreeClassifier()

In [16]:
# remove baseline assumption from the train
train.drop(columns='baseline_assumption_death', inplace=True)

In [17]:
# split our X and y
X_train = train.drop(columns='survived')
y_train = train.survived

In [18]:
# fit the model
clf1.fit(X_train, y_train)

DecisionTreeClassifier()

3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [19]:
# use the model to predict
y_pred = clf1.predict(X_train)

In [20]:
# check out the values in the predictions
pd.Series(y_pred).value_counts()

0    243
1    156
dtype: int64

In [21]:
# model score: accuracy

In [22]:
accuracy = clf1.score(X_train, y_train)

In [23]:
accuracy

0.9799498746867168

In [24]:
# confusion matrix
conf = confusion_matrix(y_train, y_pred)

In [25]:
conf

array([[236,   1],
       [  7, 155]])

In [26]:
# get the classification report
class_report = classification_report(y_train, y_pred, output_dict=True)

In [27]:
class_report

{'0': {'precision': 0.9711934156378601,
  'recall': 0.9957805907172996,
  'f1-score': 0.9833333333333334,
  'support': 237},
 '1': {'precision': 0.9935897435897436,
  'recall': 0.9567901234567902,
  'f1-score': 0.9748427672955975,
  'support': 162},
 'accuracy': 0.9799498746867168,
 'macro avg': {'precision': 0.9823915796138019,
  'recall': 0.9762853570870449,
  'f1-score': 0.9790880503144654,
  'support': 399},
 'weighted avg': {'precision': 0.9802866615732113,
  'recall': 0.9799498746867168,
  'f1-score': 0.9798860358443279,
  'support': 399}}

In [28]:
pd.DataFrame(class_report).rename(columns={'0': 'deceased', '1': 'survived'}).T

,precision,recall,f1-score,support
deceased,0.971193,0.995781,0.983333,237.00000
survived,0.993590,0.956790,0.974843,162.00000
accuracy,0.979950,0.979950,0.979950,0.97995
macro avg,0.982392,0.976285,0.979088,399.00000
weighted avg,0.980287,0.979950,0.979886,399.00000


4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [29]:
conf

array([[236,   1],
       [  7, 155]])

In [30]:
# turn our confusion matrix into a dataframe for human legibility
conf_df = pd.DataFrame(conf, columns=['predict_death', 'predict_survive'], index=['actual_death', 'actual_survive'])

In [31]:
conf_df

,predict_death,predict_survive
actual_death,236,1
actual_survive,7,155


In [32]:
# make a key for reference
rubric_df = pd.DataFrame([['true negative', 'false positive'],['false negative', 'true positive']], columns=['predict_death', 'predict_survive'], index=['actual_death', 'actual_survive'])

In [33]:
rubric_df

,predict_death,predict_survive
actual_death,true negative,false positive
actual_survive,false negative,true positive


In [34]:
joined = pd.concat([conf_df, rubric_df], axis=1)

In [35]:
joined

,predict_death,predict_survive,predict_death,predict_survive
actual_death,236,1,true negative,false positive
actual_survive,7,155,false negative,true positive


In [36]:
# crate a function to calculate these metrics
def get_metrics_binary(clf):
    '''
    get_metrics_binary takes in a confusion matrix (cnf) for a binary classifier and prints out metrics based on
    values in variables named X_train, y_train, and y_pred.
    
    return: a classification report as a transposed DataFrame
    '''
    accuracy = clf.score(X_train, y_train)
    class_report = pd.DataFrame(classification_report(y_train, y_pred, output_dict=True)).T
    conf = confusion_matrix(y_train, y_pred)
    tpr = conf[1][1] / conf[1].sum()
    fpr = conf[0][1] / conf[0].sum()
    tnr = conf[0][0] / conf[0].sum()
    fnr = conf[1][0] / conf[1].sum()
    print(f'''
    The accuracy for our model is {accuracy:.4}
    The True Positive Rate is {tpr:.3}, The False Positive Rate is {fpr:.3},
    The True Negative Rate is {tnr:.3}, and the False Negative Rate is {fnr:.3}
    ''')
    return class_report
    

In [37]:
# call our function
report_df = get_metrics_binary(clf1)


    The accuracy for our model is 0.9799
    The True Positive Rate is 0.957, The False Positive Rate is 0.00422,
    The True Negative Rate is 0.996, and the False Negative Rate is 0.0432
    


In [38]:
report_df

,precision,recall,f1-score,support
0,0.971193,0.995781,0.983333,237.00000
1,0.993590,0.956790,0.974843,162.00000
accuracy,0.979950,0.979950,0.979950,0.97995
macro avg,0.982392,0.976285,0.979088,399.00000
weighted avg,0.980287,0.979950,0.979886,399.00000


5. Run through steps 2-4 using a different max_depth value.

In [39]:
# clf2
clf2 = DecisionTreeClassifier(max_depth=3)

In [40]:
# fit the model

In [41]:
clf2.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [42]:
y_pred = clf2.predict(X_train)

6. Which model performs better on your in-sample data?

In [43]:
report_df = get_metrics_binary(clf2)


    The accuracy for our model is 0.7018
    The True Positive Rate is 0.395, The False Positive Rate is 0.0886,
    The True Negative Rate is 0.911, and the False Negative Rate is 0.605
    


In [44]:
report_df

,precision,recall,f1-score,support
0,0.687898,0.911392,0.784029,237.000000
1,0.752941,0.395062,0.518219,162.000000
accuracy,0.701754,0.701754,0.701754,0.701754
macro avg,0.720420,0.653227,0.651124,399.000000
weighted avg,0.714307,0.701754,0.676106,399.000000


7. Which model performs best on your out-of-sample data, the validate set?

In [45]:
# get predictions for our validation sets
y_val_pred_1 = clf1.predict(validate.drop(columns='survived'))
y_val_pred_2 = clf2.predict(validate.drop(columns='survived'))

In [46]:
# get validation accuracy
accuracy_v_1 = clf1.score(validate.drop(columns='survived'), validate.survived)
accuracy_v_2 = clf2.score(validate.drop(columns='survived'), validate.survived)

In [47]:
accuracy_v_1

0.6569767441860465

In [48]:
accuracy_v_2

0.7151162790697675

In [49]:
dot_data = export_graphviz(clf2, feature_names= X_train.columns, rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data)

In [50]:
graph.render('titanic_model_2_tree', view=True)

'titanic_model_2_tree.pdf'

In [51]:
dot_data = export_graphviz(clf1, feature_names= X_train.columns, rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data)

In [52]:
graph.render('titanic_model_1_tree', view=True)

'titanic_model_1_tree.pdf'

**1. Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.**

In [53]:
rf = RandomForestClassifier(n_estimators=100, bootstrap=True, class_weight=None, criterion="gini",
                            min_samples_leaf=1, max_depth=10, random_state=1414)

In [54]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=1414)

**2. Evaluate your results using the model score, confusion matrix, and classification report.**

In [55]:
print(rf.feature_importances_)

[0.34607363 0.05862528 0.05375941 0.31197635 0.02730988 0.04033282
 0.02052979 0.09365169 0.02102659 0.00819415 0.0185204 ]


In [56]:
y_rf_pred = rf.predict(X_train)

In [57]:
y_rf_pred_proba = rf.predict_proba(X_train)

In [58]:
print(f"Random forest training sample accuracy:  {rf.score(X_train, y_train):.2%}")

Random forest training sample accuracy:  93.48%


In [59]:
pd.DataFrame(confusion_matrix(y_train, y_rf_pred, labels=(0, 1)),
             index=['actual died', 'actual survived'], columns=['predicted died', 'predicted survived'])

,predicted died,predicted survived
actual died,231,6
actual survived,20,142


In [60]:
print(classification_report(y_train, y_rf_pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.95       237
           1       0.96      0.88      0.92       162

    accuracy                           0.93       399
   macro avg       0.94      0.93      0.93       399
weighted avg       0.94      0.93      0.93       399



3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.


In [61]:
print(f"Random forest validate sample accuracy:  {rf.score(X_validate, y_validate):.2%}")

NameError: name 'X_validate' is not defined

In [ ]:
4. Run through steps increasing your min_samples_leaf and decreasing your max_depth.



In [ ]:
5. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?